# GDPR Compliance — Right to Erasure & PII Masking

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/gdpr_compliance/gdpr_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/gdpr_compliance/gdpr_demo.ipynb)

## Business Scenario

Your e-commerce platform stores customer PII across Bronze, Silver, and Gold datasets.
Three compliance scenarios arise every day:

1. **A customer emails `privacy@yourcompany.com`** requesting deletion under GDPR Article 17.
2. **A dev/test environment** needs realistic-looking data, but without real names, emails, or DOBs.
3. **A regulator audit** requires a written record proving erasure was performed.

LakeLogic handles all three with a single contract annotation — `pii: true` — and no custom code.

## Business Value

| Without LakeLogic GDPR | With LakeLogic GDPR |
|---|---|
| Custom SQL per table to find PII columns | `pii: true` in the contract — one source of truth |
| Hand-crafted erasure scripts | `forget_subjects()` — one call |
| Dev environment manually sanitised | `mask_pii()` — automatic safe dataset |
| Erasure logs in spreadsheets | `generate_erasure_report()` — structured audit trail |
| Re-implemented for every new table | Reuses the same contract, scales to any dataset |

---

## Goals

1. Right-to-be-Forgotten: nullify, hash, and redact strategies
2. Batch erasure across multiple subjects
3. Whole-dataset PII masking for dev/test environments
4. Hash referential integrity check (same email → same hash across tables)
5. Generate a compliance-ready audit report


## Setup

Imports, path resolution, and loading the customer dataset.

In [ ]:
from pathlib import Path
import polars as pl

from lakelogic.core.processor import DataProcessor
from lakelogic.core.gdpr import forget_subjects, mask_pii_columns, generate_erasure_report

# ── Path resolution: works from repo root, examples/, or Colab ──────────────
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("gdpr_compliance")
CONTRACT    = EXAMPLE_DIR / "customer_contract.yaml"
DATA        = EXAMPLE_DIR / "data" / "customers.csv"

# Load the contract (gives us PII field annotations) and raw dataset
proc = DataProcessor(str(CONTRACT), engine="polars")
df   = pl.read_csv(str(DATA))

print(f"Contract:    {CONTRACT.name}")
print(f"Dataset:     {DATA.name}  ({len(df)} rows)")
pii_cols = [f.name for f in proc.contract.model.fields if f.pii]
print(f"PII columns: {pii_cols}")
print()
print(df)

## 1. Right-to-be-Forgotten — Nullify Strategy

Customer **cust_003** (Carol Williams, UK based) has emailed a deletion request
under GDPR Article 17.

The **nullify** strategy sets all `pii: true` fields to `NULL`.
Notice that `loyalty_tier` and `total_spend` are **preserved** — they are
not PII and are needed for analytics.

In [ ]:
nullified = forget_subjects(
    df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=["cust_003"],
    erasure_strategy="nullify",
)

print("After nullify erasure for cust_003:")
print(nullified)

**What happened:**
- `cust_003`'s `email`, `full_name`, `phone`, `date_of_birth`, and `shipping_address` are now `null`
- `loyalty_tier` (`Gold`) and `total_spend` (`18900.00`) are **unchanged**
- `customer_id` is kept so the row is auditable — you can prove erasure happened for that subject
- All other rows are completely untouched

## 2. Right-to-be-Forgotten — Hash Strategy

The **hash** strategy replaces PII with a one-way **SHA-256** hash.

Key benefit: **referential integrity is preserved**. If `alice@example.com` appears
in both the `customers` table and the `orders` table, both rows hash to the same
value — so cross-table joins still work after erasure.

In [ ]:
hashed = forget_subjects(
    df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=["cust_003"],
    erasure_strategy="hash",
    hash_salt="lakelogic-gdpr-2024",
)

print("After hash erasure for cust_003:")
print(hashed.select(["customer_id", "email", "full_name", "loyalty_tier", "total_spend"]))

## 3. Right-to-be-Forgotten — Redact Strategy

The **redact** strategy replaces PII with the visible marker `***REDACTED***`.
Useful when you need the empty cells to be visually obvious in a BI tool or report.

In [ ]:
redacted = forget_subjects(
    df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=["cust_003"],
    erasure_strategy="redact",
)

print("After redact erasure for cust_003:")
print(redacted.select(["customer_id", "email", "full_name", "phone", "loyalty_tier"]))

## 4. Batch Erasure — Multiple Subjects in One Call

In practice, GDPR deletion requests arrive in batches. Pass a list of `customer_id`
values and all are processed atomically in a single pass over the dataframe.

In [ ]:
batch_ids = ["cust_001", "cust_004", "cust_005"]

batch_result = forget_subjects(
    df,
    proc.contract,
    subject_column="customer_id",
    subject_ids=batch_ids,
    erasure_strategy="nullify",
)

print(f"Erased {len(batch_ids)} subjects in one call.")
print(f"Non-PII preserved — total_spend: {batch_result['total_spend'].to_list()}")
print()
print(batch_result)

## 5. Whole-Dataset PII Masking

For **dev and test environments**, you want a dataset with realistic structure
but zero real PII. `mask_pii_columns()` anonymises every row in one call —
making it safe to hand to contractors, load into staging, or use in CI.

In [ ]:
# Nullify all PII across every row
masked_null = mask_pii_columns(df, proc.contract, strategy="nullify")
print("Nullified (all rows):")
print(masked_null)
print()

In [ ]:
# Hash all PII — structure intact, values unreadable, JOINs still work
masked_hash = mask_pii_columns(df, proc.contract, strategy="hash", hash_salt="dev-env-2024")
print("Hashed (all rows):")
print(masked_hash.select(["customer_id", "email", "full_name", "loyalty_tier", "total_spend"]))

In [ ]:
# Selective masking — override the contract and target specific columns only
selective = mask_pii_columns(df, proc.contract, strategy="redact", columns=["email", "phone"])
print("Selective redact (email + phone only):")
print(selective)

## 6. Hash Referential Integrity — Cross-Table Proof

A common objection to hash-based erasure: *"will our JOINs break?"*

The answer is no — **the same plaintext always produces the same hash** (given the same salt).
The cell below proves it: `alice@example.com` appears twice in different rows;
after hashing, both produce identical values.

In [ ]:
# Simulate the same email appearing in two rows (e.g., customer table + orders table)
dup_df = pl.DataFrame({
    "customer_id":      ["c1",                 "c2",                "c1"],
    "email":            ["alice@example.com",  "bob@example.com",   "alice@example.com"],
    "full_name":        ["Alice",              "Bob",               "Alice"],
    "phone":            ["+1-555-0101",        "+1-555-0102",       "+1-555-0101"],
    "date_of_birth":    ["1985-01-01",         "1990-01-01",        "1985-01-01"],
    "shipping_address": ["123 Main St",        "456 Oak Ave",       "123 Main St"],
})

hashed_dup = mask_pii_columns(dup_df, proc.contract, strategy="hash", hash_salt="dev-env-2024")

same_person  = hashed_dup["email"][0] == hashed_dup["email"][2]   # row 0 and row 2 = Alice
diff_persons = hashed_dup["email"][0] == hashed_dup["email"][1]   # row 0 and row 1 = Alice vs Bob

print(f"Same email (Alice, rows 0 & 2)  → hashes match: {same_person}   ✔ JOINs work")
print(f"Diff email (Alice vs Bob, rows 0 & 1) → hashes match: {diff_persons}   ✔ No collisions")
print()
print(hashed_dup)

## 7. GDPR Audit Report

Every erasure operation should be logged for compliance purposes.
`generate_erasure_report()` produces a structured dictionary you can:
- Write to a compliance log table
- Emit to a SIEM or audit trail
- Save as JSON alongside the processed dataset

The `compliance_note` field directly references **GDPR Article 17**.

In [ ]:
report = generate_erasure_report(
    proc.contract,
    subject_column="customer_id",
    subject_ids=["cust_003"],
    erasure_strategy="nullify",
    affected_rows=1,
)

print("GDPR Erasure Audit Report")
print("-" * 40)
for key, value in report.items():
    print(f"  {key:<28}: {value}")

In [ ]:
# Optionally save the report as JSON for an audit trail
import json
from pathlib import Path

report_path = EXAMPLE_DIR / "data" / "erasure_report_cust_003.json"
report_path.write_text(json.dumps(report, indent=2, default=str))
print(f"Audit report saved to: {report_path}")
print(report_path.read_text())

---

## Summary

| Scenario | API |
|---|---|
| Erase a single subject | `forget_subjects(df, contract, subject_column, ["cust_003"], erasure_strategy="nullify")` |
| Batch erasure | `forget_subjects(df, contract, subject_column, ["cust_001", "cust_004"])` |
| Anonymise entire dataset | `mask_pii_columns(df, contract, strategy="hash", hash_salt="...")` |
| Target specific columns | `mask_pii_columns(df, contract, columns=["email", "phone"])` |
| Compliance audit record | `generate_erasure_report(contract, ...)` |

**All PII columns are discovered automatically** from the `pii: true` annotations
in your contract — no hardcoded column names in your pipeline code.

When you add a new PII field to the contract, it is automatically included
in all future erasure and masking operations.
